In [13]:
# https://medium.com/@smoothml/scikit-learn-predictions-on-apache-spark-3987feff44a2
df = pd.read_parquet('walmart.parquet')
df.head(1)

,weekly_sales,sid,dept,store,type,size,temperature_stdev,fuel_price_avg,fuel_price_stdev,cpi_avg,cpi_stdev,unemployment_avg,unemployment_stdev,holidayfreq
0,6,253385,5,24,1,203819,34.785,9.963135,3.499,0.268701,134.613271,2.638256,8.3645,0.126572


In [14]:
X = df[df.columns[df.columns != 'weekly_sales']]
y = df['weekly_sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

In [17]:
from sklearn import tree
from sklearn.externals import joblib

model_path = 'spark-pandas-udf-sklearn.pkl'

clf = tree.DecisionTreeClassifier(max_depth=8)
clf.fit(X_train, y_train)
joblib.dump(clf, model_name)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

['spark-pandas-udf-sklearn.pkl']

In [18]:
sdf = spark.createDataFrame(X_test)
sdf.show(1)

+------+----+-----+----+------+-----------------+----------------+----------------+-----------------+----------------+----------------+------------------+-----------------+
|   sid|dept|store|type|  size|temperature_stdev|  fuel_price_avg|fuel_price_stdev|          cpi_avg|       cpi_stdev|unemployment_avg|unemployment_stdev|      holidayfreq|
+------+----+-----+----+------+-----------------+----------------+----------------+-----------------+----------------+----------------+------------------+-----------------+
|343242|  11|   40|   1|155083| 62.9333333333333|7.10115718269444|3.45866666666667|0.643245935341482|135.711968466667|2.73987539433759|  4.68866666666667|0.591980855546306|
+------+----+-----+----+------+-----------------+----------------+----------------+-----------------+----------------+----------------+------------------+-----------------+
only showing top 1 row



In [19]:
from pyspark.sql.functions import PandasUDFType, pandas_udf

def make_predictions(sc, df, feature_cols, model_path):
    clf = sc.broadcast(joblib.load(model_path))

    @pandas_udf(returnType='int', functionType=PandasUDFType.SCALAR)
    def predict(*cols):
        X = pd.concat(cols, axis=1)
        predictions = clf.value.predict(X)
        return pd.Series(predictions)

    return df.withColumn("predictions", predict(*feature_cols))

In [20]:
pred = make_predictions(spark.sparkContext, sdf, X.columns, 'spark-pandas-udf-sklearn.pkl')
pred.show()

Py4JJavaError: An error occurred while calling o460.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7, localhost, executor driver): java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.arrow.vector.ipc.message.MessageChannelReader.readNext(MessageChannelReader.java:58)
	at org.apache.arrow.vector.ipc.ArrowStreamReader.readSchema(ArrowStreamReader.java:132)
	at org.apache.arrow.vector.ipc.ArrowReader.initialize(ArrowReader.java:181)
	at org.apache.arrow.vector.ipc.ArrowReader.ensureInitialized(ArrowReader.java:172)
	at org.apache.arrow.vector.ipc.ArrowReader.getVectorSchemaRoot(ArrowReader.java:65)
	at org.apache.spark.sql.execution.python.ArrowPythonRunner$$anon$1.read(ArrowPythonRunner.scala:162)
	at org.apache.spark.s